In [ ]:
car = read.csv("c2015.csv", header = TRUE)

In [ ]:
library(caret)
library(ranger)
library(rpart)
library(ggplot2)

In [ ]:
### Cleans and simlifies the data. Eliminates a fair amount of data to get rid of no useful variables and missing data.
car$VEH_NO = NULL
car$YEAR = NULL
car$LATITUDE = NULL
car$LONGITUD = NULL
car$ST_CASE = NULL
names(car)[11] = "target"
car$AGE = as.numeric(car$AGE)
car$TRAV_SP = as.integer(car$TRAV_SP)
car = car[car$TRAV_SP < 133,]
car = car[car$HOUR < 25, ]
car = car[car$AGE < 104, ]
car$MINUTE[is.na(car$MINUTE)] = mean(car$MINUTE, na.rm = TRUE)
car$HOUR[is.na(car$HOUR)] = mean(car$HOUR, na.rm = TRUE)
car$COUNTY = factor(car$COUNTY)
levels(car$SEX) = c("Female", "Male", "Female", "Male")
levels(car$STATE) = c("Southeast", "Northwest", "Southwest", "Southeast", "Southwest", "Midwest", "Northeast", "MidAtlantic", "MidAtlantic", "Southeast", "Southeast", "Southwest", "Northwest", "Midwest", "Midwest", "Midwest", "Midwest", "MidAtlantic", "Southeast", "Northeast", "MidAtlantic", "Northeast", "Midwest", "Midwest", "Southeast", "Midwest", "Northwest","Midwest","Southwest", "Northeast", "Northeast", "Southeast", "Northeast", "MidAtlantic","Midwest", "Midwest", "Southwest", "Northwest","MidAtlantic", "Northeast", "Southeast","Midwest", "MidAtlantic","Southwest","Southwest", "Northeast", "MidAtlantic", "Northwest", "MidAtlantic", "Midwest", "Northwest")
levels(car$MONTH) = c("Spring", "Summer", "Fall", "Winter", "Winter", "Summer", "Spring", "Winter", "Spring", "Fall", "Fall", "Summer")
levels(car$MOD_YEAR) =c("Unknown", "1930", "1930", "1930", "1930", "1930", "1930", "1940", "1940", "1940", "1940", "1940", "1940", "1950", "1950", "1950", "1950", "1950", "1950", "1950", "1960", "1960", "1960", "1960", "1960", "1960", "1960", "1960", "1960", "1970", "1970", "1970", "1970", "1970", "1970", "1970", "1970", "1970", "1970", "1980", "1980", "1980", "1980", "1980", "1980", "1980", "1980", "1980", "1980", "1990", "1990", "1990", "1990", "1990", "1990", "1990", "1990", "1990", "1990", "2000", "2000", "2000", "2000", "2000", "2000", "2000", "2000", "2000", "2000", "2010", "2010", "2010", "2010", "2010", "2010", "2010", "Unknown", "Unknown") 
levels(car$target) = c("Fatal", "Fatal", "Live", "Live", "Live", "Live", "Live", "Live")
levels(car$DEFORMED) = c("Unknown", "Disabling Damage", "Functional Damage", "Minor Damage", "No Damage", "Not Reported", "Unknown")
levels(car$MAN_COLL)  = c("Other", "Angle", "Front-to-Front", "Front-to-Rear", 
"Not a Collision with Motor Vehicle In-Transport", "Not Reported", "Other", "Rear-to-Rear", "Rear-to-Side", 
"Sideswipe", "Sideswipe", "Other")
levels(car$OWNER) = c("Unknown", "Driver (in this crash) Not Registered Owner (Other Private Owner Listed)",
"Driver (in this crash) was  Registered Owner", "Driverless/Motor Vehicle Parked/Stopped Off Roadway",                 "Not Applicable, Vehicle Not Registered", "Unknown",                                                               "Vehicle Registered as Business/Company/Government Vehicle","Vehicle Registered as Rental Vehicle",                    "Vehicle was Stolen (reported by police)" )
quick_clean(car,1)
summary(car)

In [ ]:
### Visuals comparing the target variable to all numeric variables 
ggplot(data=car)+geom_density(mapping = aes(x=car$target,color=car$STATE))+labs(x=car$target,color=car$STATE)
ggplot(data=car)+geom_density(mapping = aes(x=car$target,color=car$MONTH))+labs(x=car$target,color=car$MONTH)
ggplot(data=car)+geom_density(mapping = aes(x=car$target,color=car$SEX))+labs(x=car$target,color=car$SEX)
ggplot(data=car)+geom_density(mapping = aes(x=car$target,color=car$PER_TYP))+labs(x=car$target,color=car$PER_TYP)
ggplot(data=car)+geom_density(mapping = aes(x=car$target,color=car$SEAT_POS))+labs(x=car$target,color=car$SEAT_POS)
ggplot(data=car)+geom_density(mapping = aes(x=car$target,color=car$DRINKING))+labs(x=car$target,color=car$DRINKING)
ggplot(data=car)+geom_density(mapping = aes(x=car$target,color=car$MAN_COLL))+labs(x=car$target,color=car$MAN_COLL)
ggplot(data=car)+geom_density(mapping = aes(x=car$target,color=car$OWNER))+labs(x=car$target,color=car$OWNER)
ggplot(data=car)+geom_density(mapping = aes(x=car$target,color=car$MOD_YEAR))+labs(x=car$target,color=car$MOD_YEAR)
ggplot(data=car)+geom_density(mapping = aes(x=car$target,color=car$DEFORMED))+labs(x=car$target,color=car$DEFORMED)
ggplot(data=car)+geom_density(mapping = aes(x=car$target,color=car$DAY_WEEK))+labs(x=car$target,color=car$DAY_WEEK)
ggplot(data=car)+geom_density(mapping = aes(x=car$target,color=car$ROUTE))+labs(x=car$target,color=car$ROUTE)
ggplot(data=car)+geom_density(mapping = aes(x=car$target,color=car$LGT_COND))+labs(x=car$target,color=car$LGT_COND)
ggplot(data=car)+geom_density(mapping = aes(x=car$target,color=car$WEATHER))+labs(x=car$target,color=car$WEATHER)

In [ ]:
cars = car[sample(1:nrow(car), 1000),]
set.seed(12)
quick_model(cars)

### Accuracy = .79495 mtry 193, gini, nodes 1.

In [ ]:
myGrid = expand.grid(mtry = c(10:(ncol(cars)-1)), splitrule = c("gini","extratrees"),
                     min.node.size = c(1:7))

model <- train(target~.,data = cars, method = "ranger", 
               trControl = trainControl(method ="cv", number = 3, verboseIter = TRUE),
               tuneGrid = myGrid)
plot(model)
print(model)

### accuracy = .792 mrty = 19, slitrule = gini, node.size = 3

In [ ]:
splitIndex <- createDataPartition(adult$target, p = .70, list = FALSE, times = 1)
train <- cars[ splitIndex,]
test <- cars[-splitIndex,]
mytree <- rpart(target ~ ., data = train, method = "class")
pred1 <- predict(mytree,test, type = "class")
cm8=confusionMatrix(data = pred1, reference = test$target, positive = NULL)
cm8

### balance accuracy = .7241

In [ ]:
myControl <- trainControl(method = "cv", number = 10, summaryFunction = twoClassSummary,
classProbs = TRUE, verboseIter = TRUE)
myGrid2 = expand.grid(alpha = 0:1,lambda = seq(0.0001, 0.1, length = 10))
model2 <- train(target~.,data = cars, method = "glmnet", 
               trControl = myControl, tuneGrid = myGrid2)
print(model2)
plot(model2)

###  ROC = .8666 alpha = 1 lamda = .0223